In [ ]:
import gzip
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = []
file_path = "renttherunway_final_data.json.gz"

with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error decoding line: {e}")

df = pd.DataFrame(data)
df.head()

## Data Cleaning

In [ ]:
### Data Overview
overview = df.info()
overview

In [ ]:
### Descriptive Statistics
descriptive_stats = df.describe(include="all")
descriptive_stats

In [26]:
## Adrian Cell
df['item_id'].nunique()

5850

In [ ]:
df['category'].unique()

In [ ]:
print('test')